---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant information from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order according to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('data/dates.txt') as file:
    for line in file:
        doc.append(line)
df = pd.Series(doc)
df.head()

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
dtype: object

In [38]:
def date_sorter():
    # Your code here
    dates_extracted = df.str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4}))') # Example: 04/20/2009
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index]) # leftover indices
    dates_extracted = dates_extracted.loc[:, :'year'] # remove extra columns
    dates_extracted = pd.concat([dates_extracted, df[index_left].str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2}))')]) # Example: 4/20/09; 4/3/09
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index]) # leftover indices
    dates_extracted = dates_extracted.loc[:, :'year'] # remove extra columns
    dates_extracted = pd.concat([dates_extracted, df[index_left].str.extractall(r'(?P<origin>(?P<day>\d?\d) ?(?P<month>[a-zA-Z]{3,})\.?,? (?P<year>\d{4}))')]) # Example: 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index]) # leftover indices
    dates_extracted = dates_extracted.loc[:, :'year'] # remove extra columns
    dates_extracted = pd.concat([dates_extracted, df[index_left].str.extractall(r'(?P<origin>(?P<month>[a-zA-Z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4}))')]) # Example: Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index]) # leftover indices
    dates_extracted = dates_extracted.loc[:, :'year'] # remove extra columns

    dates_without_day = df[index_left].str.extractall('(?P<origin>(?P<month>[A-Z][a-z]{2,}),?\.? (?P<year>\d{4}))') # Example: Feb 2009; Sep 2009; Oct 2010
    dates_without_day = dates_without_day.loc[:, :'year'] # remove extra columns
    dates_without_day = pd.concat([dates_without_day, df[index_left].str.extractall(r'(?P<origin>(?P<month>\d\d?)/(?P<year>\d{4}))')]) # Example: 6/2008; 12/2009
    dates_without_day = dates_without_day.loc[:, :'year'] # remove extra columns
    dates_without_day['day'] = 1 # set 1 as day for dates without days
    dates_extracted = pd.concat([dates_extracted, dates_without_day]) # update extracted dates
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index]) # leftover indices

    dates_only_year = df[index_left].str.extractall(r'(?P<origin>(?P<year>\d{4}))') # Example: 2009; 2010
    dates_without_day = dates_without_day.loc[:, :'year'] # remove extra columns
    dates_only_year['day'] = 1 # set 1 as day for dates without days
    dates_only_year['month'] = 1 # set 1 as month for dates without months
    dates_extracted = pd.concat([dates_extracted, dates_only_year]) # update extracted dates

    dates_extracted['year'] = dates_extracted['year'].apply(lambda x: '19' + x if len(x) == 2 else x) # Example: 99 --> 1999
    dates_extracted['year'] = dates_extracted['year'].apply(lambda x: str(x)) # Set year as string

    dates_extracted['month'] = dates_extracted['month'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x) # Example: 01 --> 1
    month_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                       'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                       'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                       'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
    dates_extracted.replace({"month": month_dict}, inplace=True) # replace month name by month number
    dates_extracted['month'] = dates_extracted['month'].apply(lambda x: str(x)) # Set month as string

    dates_extracted['day'] = dates_extracted['day'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x) # Example: 01 --> 1
    dates_extracted['day'] = dates_extracted['day'].apply(lambda x: str(x)) # Set day as string

    dates_extracted['date'] = dates_extracted['month'] + '/' + dates_extracted['day'] + '/' + dates_extracted['year'] # Set date format as mm/dd/yyyy
    dates_extracted['date'] = pd.to_datetime(dates_extracted['date'])# Convert to datetime

    dates_extracted.sort_values(by='date', inplace=True) # sort df by date
    ser = pd.Series(dates_extracted.index.get_level_values(0), name="Chronological order") # Create pandas series
    
    return ser # Your answer here

In [39]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Name: Chronological order, Length: 500, dtype: int64